# Finalize Sample

In this notebook, I finalize the LSBG sample, by doing the following:

- Get the latest Galfit segmap results.
- Correct for extinction.
- Include the large ($r^{1/2} > 20''$) galaxies. 
- Reject those with $\langle \mu \rangle_e < 24.3''$ and $R_{eff} <2.5''$.

In [1]:
#Import stuff
import numpy as np 
import pandas as pd
from astropy.table import Table
from astropy.io import fits
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

### Import the Galfit results

In [2]:
Galfit_res_s = fits.open('y3_gold_2_2_lsbg_galfit_v4.6.fits')
# =========================================================
# =========================================================
# =========================================================
object_id_LSBGs = Galfit_res_s[1].data["COADD_OBJECT_ID"]
    
# Coordinates - Sextractor
RA = Galfit_res_s[1].data["RA"]
DEC = Galfit_res_s[1].data["DEC"]

# Coordinates - Galfit
RA_gf = Galfit_res_s[1].data["ALPHA_J2000"]
DEC_gf = Galfit_res_s[1].data["DELTA_J2000"]

# =========================================================
# =========================================================
# A, B image
A_IMAGE = Galfit_res_s[1].data["A_IMAGE"]
B_IMAGE = Galfit_res_s[1].data["B_IMAGE"]

# Define Ellipticity - Sextractor
Ell = 1.0 - B_IMAGE/A_IMAGE

# Axis ratio
AR = Galfit_res_s[1].data["AR"]
# Define Ellipticity - Galfit
Ell_gf = 1.0 - AR
#print(Ell-Ell_gf)

# ==========================================================
# ==========================================================
# Effective radius in g-band and error 
R_eff_g = 0.263*Galfit_res_s[1].data["RE_G"]
R_eff_g_err = 0.263*Galfit_res_s[1].data["RE_ERR_G"]
# Effective radius in r-band and error
R_eff_r = 0.263*Galfit_res_s[1].data["RE_R"]
R_eff_r_err = 0.263*Galfit_res_s[1].data["RE_ERR_R"]
# Effective radius in i-band and error
R_eff_i = 0.263*Galfit_res_s[1].data["RE_I"]
R_eff_i_err = 0.263*Galfit_res_s[1].data["RE_ERR_I"]
# ========================================================
# ========================================================

# Flux radius in g-band
flux_radius_g = Galfit_res_s[1].data["FLUX_RADIUS_G"]
# Flux radius in r-band
flux_radius_r = Galfit_res_s[1].data["FLUX_RADIUS_R"]
# Flux radius in i-band
flux_radius_i = Galfit_res_s[1].data["FLUX_RADIUS_I"]
# ========================================================
# ========================================================
# Sersic index
n_ser = Galfit_res_s[1].data["N"]
n_ser_err = Galfit_res_s[1].data["N_ERR"]

# Galfitm Magnitudes
mag_gf_g = Galfit_res_s[1].data["MAG_G"]
mag_gf_r = Galfit_res_s[1].data["MAG_R"]
mag_gf_i = Galfit_res_s[1].data["MAG_I"]


# Galfitm Magnitude errors
magerr_gf_g = Galfit_res_s[1].data["MAG_ERR_G"]
magerr_gf_r = Galfit_res_s[1].data["MAG_ERR_R"]
magerr_gf_i = Galfit_res_s[1].data["MAG_ERR_I"]

# SExtractor (MAG_AUTO) magnitudes
mag_auto_g = Galfit_res_s[1].data["MAG_AUTO_G"]
mag_auto_r = Galfit_res_s[1].data["MAG_AUTO_R"]
mag_auto_i = Galfit_res_s[1].data["MAG_AUTO_I"]

# MAG_AUTO magnitude errors
magerr_auto_g = Galfit_res_s[1].data["MAGERR_AUTO_G"]
magerr_auto_r = Galfit_res_s[1].data["MAGERR_AUTO_R"]
magerr_auto_i = Galfit_res_s[1].data["MAGERR_AUTO_I"]
# ====================================================
# ====================================================
# Chi-squared values
chi2_g = Galfit_res_s[1].data['CHI2NU_G']
chi2_r = Galfit_res_s[1].data['CHI2NU_R']
chi2_i = Galfit_res_s[1].data['CHI2NU_I']

#Galfit_res_s[1].header.keys

#### Import the Extinction Results

In [3]:
Ext_res = fits.open('Extinction.fits')
#Ext_res[1].header.keys
object_id_EXT = Ext_res[1].data["COADD_OBJECT_ID"] #The coadd 
Ext_ebv_sfd98 = Ext_res[1].data["ebv_sfd98"]

Now create a dataframe that contains the coadd_ids of all objects in the extinction catalog...

In [4]:
df_all = pd.DataFrame({'all_coadds':np.asarray(object_id_EXT)}) #Dataframe that contains the 
# coadd object IDs for all objects we have extinction for
# =======================================================================
# =======================================================================
df_LSBGs =  pd.DataFrame({'LSBG_coadds':np.asarray(object_id_LSBGs)}) #Dataframe that contains the 
# coadd object IDs for LSBGs

In [5]:
mask = df_all['all_coadds'].isin(object_id_LSBGs)
object_ids_true = object_id_EXT[mask]
print(len(object_ids_true)) 
extinction = Ext_ebv_sfd98[mask]
# =====================================
# =====================================
sort = np.argsort(object_ids_true)
ext_sort = extinction[sort]   #Use the sorted extinction to correct the magnitudes. 

21292


##### Correct the magnitudes for extinction

I will correct the magnitudes for galaxy extinction, based on the following conversions:

mag_g = mag_g - 3.186$\times$ebv_sfd98

mag_r = mag_r - 2.140$\times$ebv_sfd98

mag_i = mag_i - 1.569$\times$ebv_sfd98

In [6]:
# Correct the galfit magnitudes
mag_gf_g_cor = mag_gf_g - 3.186*ext_sort
mag_gf_r_cor = mag_gf_r - 2.140*ext_sort
mag_gf_i_cor = mag_gf_i - 1.569*ext_sort

# Correct the mag_auto magnitudes
mag_auto_g_cor = mag_auto_g - 3.186*ext_sort
mag_auto_r_cor = mag_auto_r - 2.140*ext_sort
mag_auto_i_cor = mag_auto_i - 1.569*ext_sort

### Mean and central surface brightness calculations

The mean surface brightness can be calculated from the effective radius and the magnitude as:

\begin{equation}
\left\langle \mu \right\rangle_{eff} = m + 2.5\log (2\pi R_{eff}^2)
\end{equation}

Where in $\mu$ we use the extinction-corrected apparent magnitude in each one of the three bands, $g, r, i$.

In [7]:
# mean surface brightness in g-band
mu_mean_g = mag_gf_g_cor + 2.5*np.log10(2.0*np.pi*(R_eff_g**2.0))
# mean surface brightness in r-band
mu_mean_r = mag_gf_r_cor + 2.5*np.log10(2.0*np.pi*(R_eff_r**2.0))
# mean surface brightness in i-band
mu_mean_i = mag_gf_i_cor + 2.5*np.log10(2.0*np.pi*(R_eff_i**2.0))

For this part we will need to estimate the central surface brightness $\mu_0$. This is given by:

\begin{equation}
\mu_0 = \mu_e - \frac{2.5 b}{\ln(10)}
\end{equation}

$b$ can be calculated by solving the equation:

\begin{equation}
\Gamma(2n) = 2\gamma(2n,b),
\end{equation}
where:
\begin{equation}
\gamma(2n,x) = \int_0^x e^{-t}t^{2n-1} dt
\end{equation}
the incomplete gamma function.

Now, we have to estimate $\mu_e$. To do that we use the mean surface brightness, $\langle \mu \rangle_e$, calculated above.

We have that: 
\begin{equation}
\mu_e = \langle \mu \rangle_e + 2.5\log[f(n)]
\end{equation}

Where:
\begin{equation}
f(n) = \frac{ne^b}{b^{2n}}\Gamma(2n)
\end{equation}
and
\begin{equation}
\Gamma(2n) = \int_0^\infty e^{-x} x^{2n - 1} dx
\end{equation}


The final formula for $\mu_0$ is:

\begin{equation}
\mu_0 = \langle \mu \rangle_e + 2.5\log[f(n)] - \frac{2.5 b}{\ln(10)}
\end{equation}

with the needed parameters calculated as described above.

In [8]:
# First part, calculation of b for a given n. Write a function that does that.
from scipy.special import gamma
from scipy.special import gammainc # Note that the incomplete gamma function 
from scipy.optimize import root_scalar

def funct(x, *args):
    """
    The function to be minimized
    """
    n = args[0]
    fun = 2.0*gammainc(2.0*n,x) - 1.0
    return fun
    
def b_return(n):
    """
    This function returns the parameter b for a given
    Sersic index n, solving the equations described above
    """
    
    b_ret = root_scalar(funct, args=(n), method='bisect', bracket=[0.01, 40])
    return b_ret.root
    
b_n = np.zeros(len(n_ser))


for i in range(len(b_n)):
    b_n[i] = b_return(n_ser[i])   

In [9]:
# Now calculate the function f(n)
f_n = (n_ser*np.exp(b_n)/(b_n**(2.0*n_ser)))*gamma(2.0*n_ser)

# Calculate central surface brightness for the three bands
# g-band first
mu_cent_g = mu_mean_g + 2.5*np.log10(f_n) -  2.5*b_n/np.log(10.0)
# r-band 
mu_cent_r = mu_mean_r + 2.5*np.log10(f_n) -  2.5*b_n/np.log(10.0)
# i-band
mu_cent_i = mu_mean_i + 2.5*np.log10(f_n) -  2.5*b_n/np.log(10.0)

### Now perform the selection cuts

Reject those with $R_{eff} > 2.5''$ and $\langle \mu \rangle_e < 24.3''$, as measured through the galfit parameters.

Define the cuts first.

In [10]:
cuts = (mu_mean_g>=24.3)&(R_eff_g>=2.5)

Apply this cut to all the parameters:

In [11]:
# Coadd ids
coadd_n = object_id_LSBGs[cuts]

# Coordinates - Sextractor
RA_n = RA[cuts] 
DEC_n = DEC[cuts]
# Coordinates - Galfit
RA_gf_n = RA_gf[cuts]
DEC_gf_n = DEC_gf[cuts]
# ===========================================
# ===========================================
# A, B image
A_IMAGE_n = A_IMAGE[cuts]
B_IMAGE_n = B_IMAGE[cuts]

# Ellipticity - Sextractor
Ell_n = Ell[cuts]
# Ellipticity - Galfit
Ell_gf_n = Ell_gf[cuts]
# ===========================================
# ===========================================
# Effective Radii

# g-band
R_eff_g_n = R_eff_g[cuts]
R_eff_g_err_n = R_eff_g_err[cuts]
# r-band
R_eff_r_n = R_eff_r[cuts]
R_eff_r_err_n = R_eff_r_err[cuts]
# i-band
R_eff_i_n = R_eff_i[cuts]
R_eff_i_err_n = R_eff_i_err[cuts]
# ===========================================
# Flux Radii

# g-band
flux_radius_g_n = flux_radius_g[cuts]
# r-band
flux_radius_r_n = flux_radius_r[cuts]
# i-band
flux_radius_i_n = flux_radius_i[cuts]

# ===========================================
# ===========================================
# Sersic index 
n_ser_n = n_ser[cuts]
n_ser_err_n = n_ser_err[cuts]

# Galfit magnitudes
mag_gf_g_cor_n = mag_gf_g_cor[cuts]
mag_gf_r_cor_n = mag_gf_r_cor[cuts]
mag_gf_i_cor_n = mag_gf_i_cor[cuts]


# Galfit magnitude errors
magerr_gf_g_n = magerr_gf_g[cuts]
magerr_gf_r_n = magerr_gf_r[cuts]
magerr_gf_i_n = magerr_gf_i[cuts]


# MAG_AUTO magnitudes
mag_auto_g_cor_n = mag_auto_g_cor[cuts]
mag_auto_r_cor_n = mag_auto_r_cor[cuts]
mag_auto_i_cor_n = mag_auto_i_cor[cuts]

# MAG_AUTO magnitude errors
magerr_auto_g_n = magerr_auto_g[cuts]
magerr_auto_r_n = magerr_auto_r[cuts]
magerr_auto_i_n = magerr_auto_i[cuts]
# ============================================
# ============================================

# Extinction
ext_sort_n = ext_sort[cuts]
# ============================================
# ============================================
# mean surface 
mu_mean_g_n = mu_mean_g[cuts]
mu_mean_r_n = mu_mean_r[cuts]
mu_mean_i_n = mu_mean_i[cuts]

# Central surface
mu_cent_g_n = mu_cent_g[cuts]
mu_cent_r_n = mu_cent_r[cuts]
mu_cent_i_n = mu_cent_i[cuts]

# ============================================
# ============================================
# Chi-squared 
chi2_g_n = chi2_g[cuts]
chi2_r_n = chi2_r[cuts]
chi2_i_n = chi2_i[cuts]

print(len(n_ser_n))

20983


##### Exclude remaining bad objects

Now, exclude 6 remaining bad objects (there may be more, but these were the ones I was able to find).

In [12]:
bad_ids = [128602361,
          129395781,
          149763752,
          261926262,
          273410397,
          338312228,
          199492560,
          227222314,
          227913584,
          279057265]

Keep all, except these six.

In [13]:
# First a data frame that contains all the coadds
df_LSBG_n = pd.DataFrame({'LSBG_coadds':np.asarray(coadd_n)}) 
# ================================================================
# ================================================================
# Then a data frame that contains just the bad coadds
df_bad = pd.DataFrame({'bad_coadds':np.asarray(bad_ids)}) 

mask = df_LSBG_n['LSBG_coadds'].isin(bad_ids)

Now keep just the mask

In [14]:
# Coadd object id
object_id_f = coadd_n[~mask]

# ===============================================
# ===============================================
# RA and DEC - SExtractor
RA_f = RA_n[~mask]
DEC_f = DEC_n[~mask]
# RA and DEC - Galfit
RA_gf_f = RA_gf_n[~mask]
DEC_gf_f = DEC_gf_n[~mask]

# A, B image
A_IMAGE_f = A_IMAGE_n[~mask]
B_IMAGE_f = B_IMAGE_n[~mask]

# Ellipticity - SExtractor
Ell_f = Ell_n[~mask]
# Ellipticity - Galfit
Ell_gf_f = Ell_gf_n[~mask]
# ==============================================
# ==============================================
# Effective Radii

# g-band
R_eff_g_f = R_eff_g_n[~mask]
R_eff_g_err_f = R_eff_g_err_n[~mask]
# r-band
R_eff_r_f = R_eff_r_n[~mask]
R_eff_r_err_f = R_eff_r_err_n[~mask]
# i-band
R_eff_i_f = R_eff_i_n[~mask]
R_eff_i_err_f = R_eff_i_err_n[~mask]
# ============================================
# Flux Radii

# g-band
flux_radius_g_f = flux_radius_g_n[~mask]
# r-band
flux_radius_r_f = flux_radius_r_n[~mask]
# i-band
flux_radius_i_f = flux_radius_i_n[~mask]
# ============================================
# ============================================
# Sersic index 
n_ser_f = n_ser_n[~mask]
n_ser_err_f = n_ser_err_n[~mask]

# Galfit magnitudes
mag_gf_g_cor_f = mag_gf_g_cor_n[~mask]
mag_gf_r_cor_f = mag_gf_r_cor_n[~mask]
mag_gf_i_cor_f = mag_gf_i_cor_n[~mask]

# Galfit magnitude errors
magerr_gf_g_f = magerr_gf_g_n [~mask]
magerr_gf_r_f = magerr_gf_r_n [~mask]
magerr_gf_i_f = magerr_gf_i_n [~mask]

# MAG_AUTO magnitudes
mag_auto_g_cor_f = mag_auto_g_cor_n[~mask]
mag_auto_r_cor_f = mag_auto_r_cor_n[~mask]
mag_auto_i_cor_f = mag_auto_i_cor_n[~mask]

# MAG_AUTO magnitude errors
magerr_auto_g_f = magerr_auto_g_n[~mask]
magerr_auto_r_f = magerr_auto_r_n[~mask]
magerr_auto_i_f = magerr_auto_i_n[~mask]


extinction_f = ext_sort_n[~mask]

# ============================================
# ============================================
# mean surface 
mu_mean_g_f = mu_mean_g_n[~mask]
mu_mean_r_f = mu_mean_r_n[~mask]
mu_mean_i_f = mu_mean_i_n[~mask]

# Central surface
mu_cent_g_f = mu_cent_g_n[~mask]
mu_cent_r_f = mu_cent_r_n[~mask]
mu_cent_i_f = mu_cent_i_n[~mask]

# ============================================
# ============================================
# Chi-squared values
chi2_g_f = chi2_g_n[~mask]
chi2_r_f = chi2_r_n[~mask]
chi2_i_f = chi2_i_n[~mask]

print(len(n_ser_f))

20977


### Save in a csv file

Save the ones that I have above; later I can add more.

In [15]:
from collections import OrderedDict

#LSBG_df = pd.DataFrame( OrderedDict(( ('Coadd_id', pd.Series(object_id_f)),
#                                      ('RA', pd.Series(RA_f)), ('DEC', pd.Series(DEC_f)),
#                                   ('A_IMAGE',pd.Series(A_IMAGE_f)),('B_IMAGE',pd.Series(B_IMAGE_f)),
#                                    ('R_eff_g', pd.Series(R_eff_f)),('R_eff_err_g', pd.Series(R_eff_err_f)),
#                                     ('R_eff_i', pd.Series(R_eff_i_f)),('R_eff_err_i', pd.Series(R_eff_err_i_f)),
#                                     ('n_ser', pd.Series(n_ser_f)),  ('n_ser_err', pd.Series(n_ser_err_f)),
#                                     ('mag_g', pd.Series(mag_g_cor_f)),('mag_r', pd.Series(mag_r_cor_f)),('mag_i', pd.Series(mag_i_cor_f)),
#                                     ('mag_auto_g', pd.Series(mag_auto_g_cor_f)),
#                                    ('mag_auto_r', pd.Series(mag_auto_r_cor_f)),
#                                    ('mag_auto_i', pd.Series(mag_auto_i_cor_f)),
#                                    ('extinction', pd.Series(extinction_f)))))


#LSBG_df.head()

In [16]:
#LSBG_df.to_csv('LSBG_sample.csv')

In [17]:
#coa = 199492560
#print(zip(RA_f[object_id_f==coa],DEC_f[object_id_f==coa]))

### Get random sample of 1000 LSBGs

Get Coadd_ids and RA, DEC of 1000 LSBGs

In [18]:
# First get 1000 random numbers from 1 to 20977
rands = np.random.randint(0,20977,1000) 
# Get the coadd ids of the 1000 galaxies corresponding to the 
# 1000 random integers above
random_coadd_ids = object_id_f[rands]
# Get teh ra and dec of the 1000 galaxies corresponding to the 
# 1000 random integers above
ra_rand = ra_f[rands];dec_rand = dec_f[rands]

# Create a dataframe and save it to a csv file
random_LSBG_df = pd.DataFrame({'coadd_ids': pd.Series(random_coadd_ids)},
                             {'ra': pd.Series(ra_rand)},
                             {'dec': pd.Series(dec_rand)})
#random_LSBG_df.head(10)
random_LSBG_df.to_csv('random_LSBGs_coadd_ra_dec.csv')


NameError: name 'ra_f' is not defined

### Create final catalog

Here create the final catalog for the publication.

In [ ]:
LSBG_df = pd.DataFrame( OrderedDict(( 
    ('coadd_object_id', pd.Series(object_id_f)),
    ('ra_se', pd.Series(RA_f)), ('dec_se', pd.Series(DEC_f)),
    ('ra_gfm', pd.Series(RA_gf_f)), ('dec_gfm', pd.Series(DEC_gf_f)),
    ('ebv_sfd98', pd.Series(extinction_f)),
    ('ell_se',pd.Series(Ell_f)),('ell_gfm',pd.Series(Ell_gf_f)),
    ('mag_auto_g_corr', pd.Series(mag_auto_g_cor_f)),
    ('magerr_auto_g', pd.Series(magerr_auto_g_f)),
    ('mag_auto_r_corr', pd.Series(mag_auto_r_cor_f)),
    ('magerr_auto_r', pd.Series(magerr_auto_r_f)),
    ('mag_auto_i_corr', pd.Series(mag_auto_i_cor_f)),
    ('magerr_auto_i', pd.Series(magerr_auto_i_f)),
    ('flux_radius_g_asec', pd.Series(flux_radius_g_f)),
    ('flux_radius_r_asec', pd.Series(flux_radius_r_f)),
    ('flux_radius_i_asec', pd.Series(flux_radius_i_f)),
    ('mag_gfm_g_corr', pd.Series(mag_gf_g_cor_f)),
    ('magerr_gfm_g', pd.Series(magerr_gf_g_f)),
    ('mag_gfm_r_corr', pd.Series(mag_gf_r_cor_f)),
    ('magerr_gfm_r', pd.Series(magerr_gf_r_f)),
    ('mag_gfm_i_corr', pd.Series(mag_gf_i_cor_f)),
    ('magerr_gfm_i', pd.Series(magerr_gf_i_f)),
    ('n', pd.Series(n_ser_f)), ('n_err', pd.Series(n_ser_err_f)),
    ('R_eff_g', pd.Series(R_eff_g_f)),('R_eff_err_g', pd.Series(R_eff_g_err_f)),
    ('R_eff_r', pd.Series(R_eff_r_f)),('R_eff_err_r', pd.Series(R_eff_r_err_f)),
    ('R_eff_i', pd.Series(R_eff_i_f)),('R_eff_err_i', pd.Series(R_eff_i_err_f)),
    ('chisq_loc_g', pd.Series(chi2_g_f)),
    ('chisq_loc_r', pd.Series(chi2_r_f)),
    ('chisq_loc_i', pd.Series(chi2_i_f)),
    ('mu_mean_g', pd.Series(mu_mean_g_f)), ('mu_mean_r', pd.Series(mu_mean_r_f)),
    ('mu_mean_i', pd.Series(mu_mean_i_f)),
    ('mu_0_g',pd.Series(mu_cent_g_f)),('mu_0_r',pd.Series(mu_cent_r_f)),
    ('mu_0_i',pd.Series(mu_cent_i_f))


)))

In [ ]:
#LSBG_df.head()

In [ ]:
LSBG_df.to_csv('LSBG_catalog.csv')

In [ ]:
#Galfit_res_s = fits.open('y3_gold_2_2_lsbg_galfit_v4.6.fits')
#Galfit_res_s[1].header.keys

In [ ]:
#DEC = Galfit_res_s[1].data["DELTA_J2000"]

In [ ]:
ral = 28.6723
decl = - 0.1432
alpha = 0.2
box = (RA_f>(ral-alpha))&(RA_f<(ral+alpha))&(DEC_f>(decl-alpha))&(DEC_f<(decl+alpha))
print(RA_f[box],DEC_f[box])